# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

Data

In [14]:
df = pd.read_csv("Dataframes/analysts.csv")
#X_var_names = ["Earnings Per Share – Coefficient of Variation", "Number of Analysts", "Recommendation - Mean (1-5)", "Recommendation change", "Price Target - Mean", "Price Target - Standard Deviation"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Recommendation change,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.00000,3.625620,7.0,3.00000,3.000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.482,-0.12500,2.046169,7.0,2.87500,3.000,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.00000,1.551601,8.0,2.87500,2.875,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,8.095,-0.25000,2.775994,7.0,2.62500,2.875,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,1.471,-0.29167,3.204412,8.0,2.33333,2.625,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.00000,2.836295,8.0,2.30000,2.300,545.28571,50.48823
20116,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.00000,3.269867,8.0,2.30000,2.300,571.00000,41.33833
20117,POOL.OQ,2022-04-01,Retailing,34.342,34.342,-0.30000,6.630736,9.0,2.00000,2.300,519.88889,37.49206
20118,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.20000,3.625249,9.0,2.20000,2.000,435.37500,57.57373


In [15]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [16]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Recommendation change,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.00000,3.625620,7.0,3.00000,3.000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.482,-0.12500,2.046169,7.0,2.87500,3.000,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.00000,1.551601,8.0,2.87500,2.875,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,8.095,-0.25000,2.775994,7.0,2.62500,2.875,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,1.471,-0.29167,3.204412,8.0,2.33333,2.625,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.00000,2.836295,8.0,2.30000,2.300,545.28571,50.48823
20116,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.00000,3.269867,8.0,2.30000,2.300,571.00000,41.33833
20117,POOL.OQ,2022-04-01,Retailing,34.342,34.342,-0.30000,6.630736,9.0,2.00000,2.300,519.88889,37.49206
20118,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.20000,3.625249,9.0,2.20000,2.000,435.37500,57.57373


In [17]:
#other way to deal with outlierrs: removing only 99th percentile outliers
'''
# Calculate the 99th percentile
pct_99 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99)

# Remove the data points above the 99th percentile
df_clean = df[df['Earnings Per Share - Actual Surprise'] <= pct_99].copy()

df_clean
'''

"\n# Calculate the 99th percentile\npct_99 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99)\n\n# Remove the data points above the 99th percentile\ndf_clean = df[df['Earnings Per Share - Actual Surprise'] <= pct_99].copy()\n\ndf_clean\n"

Standardising

In [18]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Earnings Per Share – Coefficient of Variation"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
rescale["Number of Analysts"] = MinMaxScaler().fit_transform(np.array(rescale["Number of Analysts"]).reshape(-1,1))
rescale["Recommendation change"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation change"]).reshape(-1,1))
rescale["Price Target - Mean"] = MinMaxScaler().fit_transform(np.array(rescale["Price Target - Mean"]).reshape(-1,1))
rescale["Price Target - Standard Deviation"] = MinMaxScaler().fit_transform(np.array(rescale["Price Target - Standard Deviation"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Recommendation change,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.538462,0.534474,0.125000,3.00000,3.000,0.005786,0.002079
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.480770,0.533129,0.125000,2.87500,3.000,0.006891,0.003330
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.538462,0.532708,0.145833,2.87500,2.875,0.007299,0.003658
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.423078,0.533751,0.125000,2.62500,2.875,0.007756,0.002930
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.403846,0.534116,0.145833,2.33333,2.625,0.008917,0.002908
...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.538462,0.533802,0.145833,2.30000,2.300,0.091403,0.042855
20116,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.538462,0.534171,0.145833,2.30000,2.300,0.095726,0.035088
20117,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.400001,0.537034,0.166667,2.00000,2.300,0.087134,0.031824
20118,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.630770,0.534474,0.166667,2.20000,2.000,0.072926,0.048869


In [19]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [20]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
AVY.N      2013-01-01                                11.178   
           2013-04-01                                 2.482   
           2013-07-01                                 1.068   
           2013-10-01                                 8.095   
           2014-01-01                                 1.471   
...                                                     ...   
POOL.OQ    2021-10-01                                17.194   
           2022-01-01                                40.267   
           2022-04-01                                34.342   
           2022-07-01                                 1.503   
           2022-10-01                                 4.196   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                        11.178   
           2013-04-01                                         2.482   
           2013-07-01                                         1.068   
           2013-10-01                                         8.095   
           2014-01-01                                         1.471   
...                                                             ...   
POOL.OQ    2021-10-01                                        17.194   
           2022-01-01                                        40.267   
           2022-04-01                                        34.342   
           2022-07-01                                         1.503   
           2022-10-01                                         4.196   

                       Recommendation change  \
Instrument Date                                
AVY.N      2013-01-01               0.538462   
           2013-04-01               0.480770   
           2013-07-01               0.538462   
           2013-10-01               0.423078   
           2014-01-01               0.403846   
...                                      ...   
POOL.OQ    2021-10-01               0.538462   
           2022-01-01               0.538462   
           2022-04-01               0.400001   
           2022-07-01               0.630770   
           2022-10-01               0.572030   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
AVY.N      2013-01-01                                       0.534474   
           2013-04-01                                       0.533129   
           2013-07-01                                       0.532708   
           2013-10-01                                       0.533751   
           2014-01-01                                       0.534116   
...                                                              ...   
POOL.OQ    2021-10-01                                       0.533802   
           2022-01-01                                       0.534171   
           2022-04-01                                       0.537034   
           2022-07-01                                       0.534474   
           2022-10-01                                       0.534660   

                       Number of Analysts  Recommendation - Mean 

FE Panel Regression with **Actual** Surprise Values

In [21]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Recommendation change":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,0.0028
Estimator:,PanelOLS,R-squared (Between):,0.0103
No. Observations:,17708,R-squared (Within):,0.0040
Date:,"Thu, Mar 02 2023",R-squared (Overall):,0.0006
Time:,18:19:35,Log-likelihood,-7.4e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,7.8913
Entities:,502,P-value,0.0000
Avg Obs:,35.275,Distribution:,"F(6,17162)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,5.6969


FE Panel Regression with **Absolute** Surprise Values

In [22]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Recommendation change":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0142
Estimator:,PanelOLS,R-squared (Between):,0.0291
No. Observations:,17708,R-squared (Within):,0.0145
Date:,"Thu, Mar 02 2023",R-squared (Overall):,0.0095
Time:,18:19:40,Log-likelihood,-6.916e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,41.282
Entities:,502,P-value,0.0000
Avg Obs:,35.275,Distribution:,"F(6,17162)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,31.058


By Industry: example tech industry

In [24]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise"]
X = group.loc[:, "Recommendation change":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_16896/2321553649.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_16896/2321553649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,0.0232
Estimator:,PanelOLS,R-squared (Between):,-0.0839
No. Observations:,603,R-squared (Within):,0.0282
Date:,"Thu, Mar 02 2023",R-squared (Overall):,0.0090
Time:,18:20:04,Log-likelihood,-2213.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.1252
Entities:,19,P-value,0.0490
Avg Obs:,31.737,Distribution:,"F(6,538)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,2.1000
